## Hello World SC
#### 07.1 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-09

* Create and deploy our first contract
* Write global state

## Setup
See notebook 04.1, loading `algo_util.py`, the five accounts and the Purestake credentials
* Consider hiding the code below

In [8]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [9]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json
import base64
import datetime

In [10]:
from pyteal import *

In [11]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 26072192


In [12]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

VK6CCXY4IFHIJAVMRVS543LJQEQKOJO6YQ4DZNV3D2XJI4ETYBN5354EQU
CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM
BY5K2AYO7R3G66ICY6SJ2JFVLRMIX677EAEEKDBTJZGP6Q4JVNZRDXDBKA


## A first stateful smart contract
The stateful smart contract consists of two parts
* The `approval_program` that handles everything except opt out
* The `clear_state_program` that handles the opt out

## Our first project: HelloWorld
* Idea: just write "Hello World" into the global state
* Do not do anything else

#### Step 1: Define Approval program
This is the program that handles all interactions except opt out:
* Creation of the SC `handle_creation`
* Opt-in of individual users `handle_register`
* Interactions (calls) of individual users `handle_interact`


In [13]:
handle_creation = Seq(
    [
        # Initialize a GLOBAL variable
        #                      key ---------- value
        App.globalPut( Bytes("Note"), Bytes("Hello world!") ),
        # Return "OK"
        Return(Int(1))        
    ]
)

handle_optin = Return ( Int(1) )                  # Not doing anything, returning "OK"           

handle_closeout = Return( Int(1) )                # Not doing anything, returning "OK"  

handle_updateapp = Return( Int(0) )               # Always FALSE ... updating not allowed

handle_deleteapp = Return(
    Txn.sender() == Global.creator_address()    # only TRUE if delete request is made by creator
)

# handle interaction
handle_noop  = Return ( Int(1) )                   # Not doing anything, returning "OK"           

In [14]:
hello_approval_pyteal = Cond(
    [Txn.application_id() == Int(0),              handle_creation],
    [Txn.on_completion()  == OnComplete.OptIn,    handle_optin],
    [Txn.on_completion()  == OnComplete.CloseOut, handle_closeout],
    [Txn.on_completion()  == OnComplete.UpdateApplication, handle_updateapp],
    [Txn.on_completion()  == OnComplete.DeleteApplication, handle_deleteapp],
    [Txn.on_completion()  == OnComplete.NoOp, handle_noop],
)

#### Compile PyTEAL -> TEAL
* Notice the `Mode.Application` (was `Mode.Signature`)

In [15]:
hello_approval_teal = compileTeal(hello_approval_pyteal,mode=Mode.Application, version=5)
print(hello_approval_teal)

#pragma version 5
txn ApplicationID
int 0
==
bnz main_l12
txn OnCompletion
int OptIn
==
bnz main_l11
txn OnCompletion
int CloseOut
==
bnz main_l10
txn OnCompletion
int UpdateApplication
==
bnz main_l9
txn OnCompletion
int DeleteApplication
==
bnz main_l8
txn OnCompletion
int NoOp
==
bnz main_l7
err
main_l7:
int 1
return
main_l8:
txn Sender
global CreatorAddress
==
return
main_l9:
int 0
return
main_l10:
int 1
return
main_l11:
int 1
return
main_l12:
byte "Note"
byte "Hello world!"
app_global_put
int 1
return


#### Step 1b: Define Clear State program
* This program handles forced opt-outs

In [16]:
hello_clear_pyteal =  Return(Int(1))    # not doing anything

In [17]:
hello_clear_teal = compileTeal(hello_clear_pyteal,mode=Mode.Application, version=3)
print(hello_clear_teal)

#pragma version 3
int 1
return


#### Compile TEAL -> Bytecode
This is slightly different ... we need one additional step for Byte-encoding

In [18]:
hello_approval_b64 = algod_client.compile(hello_approval_teal)
Hello_Approval =  base64.b64decode(hello_approval_b64['result'])

hello_clear_b64 = algod_client.compile(hello_clear_teal)
Hello_Clear =  base64.b64decode(hello_clear_b64['result'])

## Deploy Smart Contract

##### Bob deploys the smart contract
* Reserve global storage with `StateSchema`
* New command `ApplicationCreateTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCreateTxn)

In [19]:
# Step 1: Prepare the transaction
sp = algod_client.suggested_params()

# How much space do we need?
global_ints = 0    # no numeric variable
global_bytes = 1   # for "Note"
hello_global_schema = transaction.StateSchema(global_ints, global_bytes)

local_ints = 0     # No local variables
local_bytes = 0    # ...
hello_local_schema = transaction.StateSchema(local_ints, local_bytes)

txn = transaction.ApplicationCreateTxn(
      sender = Bob['public'],              # <-- sender public
      sp = sp,                             # <-- sp
      on_complete = 0,                     # <-- when finished do nothing
      approval_program = Hello_Approval,   # <-- approval program 
      clear_program = Hello_Clear,         # <-- clear program 
      global_schema = hello_global_schema, # <-- reserve global space 
      local_schema = hello_local_schema    # <-- reserve local space
    )

In [20]:
# Step 2: sign transaction
stxn = txn.sign(Bob['private'])

# Step 3: send
txid=algod_client.send_transactions([stxn])

# Step 4: wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26072194.
Waiting for round 26072194 to finish.
Waiting for round 26072195 to finish.
Transaction E6LGOCLYT4NNWZSX5J4HTNTNCBXXIKHHNA7SBTQCWDE6MOVVRDHQ confirmed in round 26072196.


In [21]:
txinfo

{'application-index': 147390674,
 'confirmed-round': 26072196,
 'global-state-delta': [{'key': 'Tm90ZQ==',
   'value': {'action': 1, 'bytes': 'SGVsbG8gd29ybGQh'}}],
 'pool-error': '',
 'txn': {'sig': 'wWTfvFGiq57Rw5DV4xozoRs7kwdXwfuiSj0VRQfpwCvnOSPt5siTXwBGkV3zA91jcfo1tPoBFipkHQq6We+5BA==',
  'txn': {'apap': 'BSACAQAxGCMSQAA1MRkiEkAALDEZgQISQAAiMRmBBBJAABgxGYEFEkAACjEZIxJAAAEAIkMxADIJEkMjQyJDIkOABE5vdGWADEhlbGxvIHdvcmxkIWciQw==',
   'apgs': {'nbs': 1},
   'apsu': 'AyABASJD',
   'fee': 1000,
   'fv': 26072194,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 26073194,
   'snd': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM',
   'type': 'appl'}}}

In [22]:
app_id = txinfo["application-index"]
print("Created new app-id:", app_id)

Created new app-id: 147390674


## The Smart Contract is now deployed
* And there is alreasdy something to see

In [23]:
format_state(read_global_state(algod_client,app_id))

{'Note': 'Hello world!'}

In [24]:
# Program code immediately visible on the web
print('https://testnet.algoexplorer.io/application/{}'.format(app_id))

https://testnet.algoexplorer.io/application/147390674


## Using the Smart Contract (1): Users opt-in
* Nothing will happen, but we can test everything
* Using `ApplicationOptInTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationOptInTxn)

In [25]:
user = Alice

# Step 1: prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationOptInTxn(user['public'], sp, app_id)

# Step 2: sign transaction
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: await confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26072195.
Waiting for round 26072195 to finish.
Waiting for round 26072196 to finish.
Waiting for round 26072197 to finish.
Transaction COY3MY7W6RDXRXZWHYAIRU7WL2KXNDGTAX4BYGR6MOERUASDNUUA confirmed in round 26072198.


#### Watch the state of the contract evolve

In [26]:
format_state(read_global_state(algod_client,app_id))

{'Note': 'Hello world!'}

#### Now try
* For Alice to opt-in twice (not allowed)
* For Bob to opt-in
* For Charlie to opt in

## Users can call the Smart contract ("make a visit")
* Strangley, calling a Smart Contact is done with a `ApplicationNoOpTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationNoOpTxn)

In [27]:
user = Bob

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationNoOpTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26072198.
Waiting for round 26072198 to finish.
Waiting for round 26072199 to finish.
Transaction I6UCUKNYYTMDBSNAZU3QGM2J5QF65HQ7AG27TCEZWJCKNQBGOQ4Q confirmed in round 26072200.


## Users close out (leave) App
* With a `ApplicationCloseOutTxn`
* See [here](https://py-algorand-sdk.readthedocs.io/en/latest/algosdk/future/transaction.html#algosdk.future.transaction.ApplicationCloseOutTxn)

In [28]:
user = Alice

# Step 1: prepare
sp = algod_client.suggested_params()
txn = transaction.ApplicationCloseOutTxn(user['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(user['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step 4: wait for condfirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  26072200.
Waiting for round 26072200 to finish.
Waiting for round 26072201 to finish.
Transaction RTPB7WLQHECYFTRY53DVKT43JIANH3R2RLPLA3XZ3XAY2BDGWNXA confirmed in round 26072202.


#### Now try
* For Alice to opt-out twice (not allowed)
* For Charlie to opt out

## Deleting the app
* Rather important, because an address can only create **10 apps**
* App can be deleted by creator

In [29]:
creator = Bob

# Step 1: Prepare transaction
sp = algod_client.suggested_params()
txn = transaction.ApplicationDeleteTxn(creator['public'], sp, app_id)

# Step 2: sign
stxn = txn.sign(creator['private'])

# Step 3: send
txid = algod_client.send_transactions([stxn])

# Step4 : wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

# display results
print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])

Current round is  26072202.
Waiting for round 26072202 to finish.
Waiting for round 26072203 to finish.
Transaction W7UBB2V5PJFS5CIVSQZ6DGNMDZEVEIIV2FSHRHXHPPPEKABUOU5Q confirmed in round 26072204.
Deleted app-id: 147390674


## Appendix: Some helper functions
* Also part of `algo_util.py`
* Shown here for completeness

In [30]:
def read_global_state(client, app_id):
    # reads an app's global state
    return  algod_client.application_info(app_id)["params"]["global-state"]

def format_state(state):
    # formats the state (local/global) nicely 
    formatted = {}
    textvariables = {'Info','Note'}        # <---- update this! (List of Text variables in SC)
    for item in state:
        key = base64.b64decode(item["key"]).decode("utf-8")
        value = item["value"]
        if value["type"] == 1:
            if key in textvariables:                 # Format text variables
                formatted_value = base64.b64decode(value["bytes"]).decode("utf-8")
            else:                                    # Format addresses
                formatted_value = base64.b32encode(base64.b64decode(value["bytes"]))
            formatted[key] = formatted_value
        else:
            formatted[key] = value["uint"]
    return formatted

## Appendix: two functions for cleaning up
* One address can only create up to 10 apps
* Use this code **carefully** for clearing apps

In [31]:
# For users: clear private state
def clear_app(client, private_key, app_id):
    sender = account.address_from_private_key(private_key)
    sp = client.suggested_params()
    txn = transaction.ApplicationClearStateTxn(sender, sp, app_id)
    stxn = txn.sign(private_key)
    txid = client.send_transactions([stxn])
    txinfo = wait_for_confirmation(algod_client, txid)
    print("Cleared app-id:", txinfo["txn"]["txn"]["apid"])

# For creators: kill the app
def delete_app(client, private_key, app_id):
    sender = account.address_from_private_key(private_key)
    sp = client.suggested_params()
    txn = transaction.ApplicationDeleteTxn(sender, sp, app_id)
    stxn = txn.sign(private_key)
    txid = client.send_transactions([stxn])
    txinfo = wait_for_confirmation(algod_client, txid)
    print("Deleted app-id:", txinfo["txn"]["txn"]["apid"])

In [32]:
# get a list of all created apps
applist = algod_client.account_info(Bob['public'])['created-apps']
appidlist = [app['id'] for app in applist]
print(appidlist)

[76756935, 76787288, 82121140, 83037397, 85407465]


In [33]:
# delete a specific app
app_id=71641559
delete_app(algod_client, Bob['private'], app_id)

AlgodHTTPError: TransactionPool.Remember: transaction XMRX3IDJRHS7AEKMN2WVEBLRGPBTJ66BAGL3SOZK7KHBOKOX73QA: only clearing out is supported for applications that do not exist

In [ ]:
# Remember the App is not gone from the blockchain
# It is marked "deleted"
print('https://testnet.algoexplorer.io/application/{}'.format(app_id))